In [33]:
from utils_metrics import get_entities_bio, f1_score, classification_report
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
import torch
import time
import math
import pandas as pd

In [2]:
class InputExample():
    def __init__(self, words, labels):
        self.words = words
        self.labels = labels

def template_entity(words, input_TXT, start):
    template_list = [" is an AerospaceManufacturer entity",
                     " is an AnatomicalStructure entity",
                     " is an Artist entity",
                     " is an ArtWork entity",
                     " is an Athlete entity",
                     " is a CarManufacturer entity",
                     " is a Cleric entity",
                     " is a Clothing entity",
                     " is a Disease entity",
                     " is a Drink entity",
                     " is a Facility entity",
                     " is a HumanSettlement entity",
                     " is a MedicalProcedure entity",
                     " is a Medication/Vaccine entity",
                     " is a MusicalGRP entity",
                     " is a MusicalWork entity",
                     " is an ORG entity",
                     " is an OtherLOC entity",
                     " is an OtherPER entity",
                     " is an OtherPROD entity",
                     " is a Politician entity",
                     " is a PrivateCorp entity",
                     " is a PublicCorp entity",
                     " is a Scientist entity",
                     " is a Software entity",
                     " is a SportsGRP entity",
                     " is a SportsManager entity",
                     " is a Station entity",
                     " is a Symptom entity",
                     " is a Vehicle entity",
                     " is a VisualWork entity",
                     " is a WrittenWork entity"
                    ]
    entity_dict = {0:'AerospaceManufacturer',
                   1:'AnatomicalStructure',
                   2:'Artist',
                   3:'ArtWork',
                   4:'Athlete',
                   5:'CarManufacturer',
                   6:'Cleric',
                   7:'Clothing',
                   8:'Disease',
                   9:'Drink',
                   10:'Facility',
                   11:'Food',
                   12:'HumanSettlement',
                   13:'MedicalProcedure',
                   14:'Medication/Vaccine',
                   15:'MusicalGRP',
                   16:'MusicalWork',
                   17:'ORG',
                   18:'OtherLOC',
                   19:'OtherPER',
                   20:'OtherPROD',
                   21:'Politician',
                   22:'PrivateCorp',
                   23:'PublicCorp',
                   24:'Scientist',
                   25:'Software',
                   26:'SportsGRP',
                   27:'SportsManager',
                   28:'Station',
                   29:'Symptom',
                   30:'Vehicle',
                   31:'VisualWork',
                   32:'WrittenWork'}
    num_entities = len(template_list)
    
    # input text -> template
    words_length = len(words)
    words_length_list = [len(i) for i in words]
    input_TXT = [input_TXT]*(num_entities*words_length)

    input_ids = tokenizer(input_TXT, return_tensors='pt')['input_ids']
    model.to(device)
    
    temp_list = []
    for i in range(words_length):
        for j in range(len(template_list)):
            temp_list.append(words[i]+template_list[j])

    output_ids = tokenizer(temp_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
    output_ids[:, 0] = 2
    output_length_list = [0]*num_entities*words_length


    for i in range(len(temp_list)//num_entities):
        base_length = ((tokenizer(temp_list[i * num_entities], return_tensors='pt', padding=True, truncation=True)['input_ids']).shape)[1] - 4
        output_length_list[i*num_entities:i*num_entities+ num_entities] = [base_length]*num_entities
        output_length_list[i*num_entities+4] += 1

    score = [1]*num_entities*words_length
    with torch.no_grad():
        output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids[:, :output_ids.shape[1] - 2].to(device))[0]
        for i in range(output_ids.shape[1] - 3):
            # print(input_ids.shape)
            logits = output[:, i, :]
            logits = logits.softmax(dim=1)
            # values, predictions = logits.topk(1,dim = 1)
            logits = logits.to('cpu').numpy()
            # print(output_ids[:, i+1].item())
            for j in range(0, num_entities*words_length):
                if i < output_length_list[j]:
                    score[j] = score[j] * logits[j][int(output_ids[j][i + 1])]

    end = start+(score.index(max(score))//num_entities)
        # score_list.append(score)
    return [start, end, entity_dict[(score.index(max(score))%num_entities)], max(score)] #[start_index,end_index,label,score]



def prediction(input_TXT):
    input_TXT_list = input_TXT.split(' ')

    entity_list = []
    for i in range(len(input_TXT_list)):
        words = []
        for j in range(1, min(9, len(input_TXT_list) - i + 1)):
            word = (' ').join(input_TXT_list[i:i+j])
            words.append(word)

        entity = template_entity(words, input_TXT, i) #[start_index,end_index,label,score]
        if entity[1] >= len(input_TXT_list):
            entity[1] = len(input_TXT_list)-1
        if entity[2] != 'O':
            entity_list.append(entity)
    i = 0
    if len(entity_list) > 1:
        while i < len(entity_list):
            j = i+1
            while j < len(entity_list):
                if (entity_list[i][1] < entity_list[j][0]) or (entity_list[i][0] > entity_list[j][1]):
                    j += 1
                else:
                    if entity_list[i][3] < entity_list[j][3]:
                        entity_list[i], entity_list[j] = entity_list[j], entity_list[i]
                        entity_list.pop(j)
                    else:
                        entity_list.pop(j)
            i += 1
    label_list = ['O'] * len(input_TXT_list)

    for entity in entity_list:
        label_list[entity[0]:entity[1]+1] = ["I-"+entity[2]]*(entity[1]-entity[0]+1)
        label_list[entity[0]] = "B-"+entity[2]
    return label_list

def cal_time(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [3]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('./outputs/best_model')
model.eval()
model.config.use_cache = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model.config

BartConfig {
  "_name_or_path": "./outputs/best_model",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embedd

In [5]:
score_list = []
file_path = './train_dev/en-dev.conll'
guid_index = 1
examples = []
with open(file_path, "r", encoding="utf-8") as f:
    words = []
    labels = []
    for line in f:
        if line.startswith("-DOCSTART-") or line.startswith('#') or line == "" or line == "\n":
            if words:
                examples.append(InputExample(words=words, labels=labels))
                words = []
                labels = []
        else:
            splits = line.split(" ")
            words.append(splits[0])
            if len(splits) > 1:
                labels.append(splits[-1].replace("\n", ""))
            else:
                # Examples could have no label for mode = "test"
                labels.append("O")
    if words:
        examples.append(InputExample(words=words, labels=labels))

In [ ]:
trues_list = []
preds_list = []
str = ' '
num_01 = len(examples)
num_point = 0
start = time.time()
for example in examples:
    sources = str.join(example.words)
    preds_list.append(prediction(sources))
    trues_list.append(example.labels)
    print('%d/%d (%s)'%(num_point+1, num_01, cal_time(start)))
    print(example.words)
    print('Pred:', preds_list[num_point])
    print('Gold:', trues_list[num_point])
    num_point += 1


true_entities = get_entities_bio(trues_list)
pred_entities = get_entities_bio(preds_list)
results = {
    "f1": f1_score(true_entities, pred_entities)
}
print(classification_report(true_entities, pred_entities))
print(results["f1"])
for num_point in range(len(preds_list)):
    preds_list[num_point] = ' '.join(preds_list[num_point]) + '\n'
    trues_list[num_point] = ' '.join(trues_list[num_point]) + '\n'
with open('./pred.txt', 'w') as f0:
    f0.writelines(preds_list)
with open('./gold.txt', 'w') as f0:
    f0.writelines(trues_list)

1/871 (0m 3s)
['eli', 'lilly', 'founder', 'president', 'of', 'pharmaceutical', 'company', 'eli', 'lilly', 'and', 'company']
Pred: ['B-OtherLOC', 'I-OtherLOC', 'B-OtherLOC', 'B-OtherLOC', 'O', 'O', 'B-PrivateCorp', 'B-OtherLOC', 'I-OtherLOC', 'B-Politician', 'I-Politician']
Gold: ['B-OtherPER', 'I-OtherPER', 'O', 'O', 'O', 'O', 'O', 'B-PublicCorp', 'I-PublicCorp', 'I-PublicCorp', 'I-PublicCorp']
2/871 (0m 10s)
['christoph', 'haberland', 'designed', 'a', 'new', 'marble', 'pulpit', 'for', 'the', 'church', 'which', 'was', 'built', 'in', 'italy', 'in', '1793', '.']
Pred: ['B-OtherLOC', 'I-OtherLOC', 'B-OtherLOC', 'B-Food', 'O', 'O', 'B-OtherPER', 'B-Food', 'B-Food', 'B-Facility', 'B-Food', 'B-Food', 'B-Facility', 'O', 'B-Food', 'B-Food', 'B-VisualWork', 'B-Food']
Gold: ['B-OtherPER', 'I-OtherPER', 'O', 'O', 'O', 'O', 'B-OtherPROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-HumanSettlement', 'O', 'O', 'O']
3/871 (0m 12s)
['he', 'was', 'succeeded', 'as', 'chancellor', 'by', 'sir', 'frank', 'kitto

In [26]:
# import re
# gts = []
# for x in trues_list:
#     gts.extend(re.sub('\n','',x).split(' '))

In [28]:
# preds = []
# for x in preds_list:
#     preds.extend(re.sub('\n','',x).split(' '))

In [31]:
# print(classification_report(gts, preds))

                         precision    recall  f1-score   support

B-AerospaceManufacturer       0.21      0.70      0.32        10
  B-AnatomicalStructure       0.14      0.94      0.24        17
              B-ArtWork       0.16      0.62      0.26        13
               B-Artist       0.36      0.89      0.51       212
              B-Athlete       0.20      0.82      0.32        79
      B-CarManufacturer       0.25      0.85      0.39        13
               B-Cleric       0.14      0.20      0.16        15
             B-Clothing       0.13      1.00      0.23        10
              B-Disease       0.06      0.72      0.12        18
                B-Drink       0.08      1.00      0.16        11
             B-Facility       0.19      0.71      0.30        52
                 B-Food       0.00      0.05      0.00        19
      B-HumanSettlement       0.00      0.00      0.00       109
     B-MedicalProcedure       0.00      0.00      0.00        13
   B-Medication/Vaccine 

/home/tranthh/anaconda3/envs/template/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tranthh/anaconda3/envs/template/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tranthh/anaconda3/envs/template/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,